In [5]:
from pyspark import SparkContext,SQLContext
from graphframes import *
sc = SparkContext.getOrCreate()
sc.addPyFile('/Users/neerajpadarthi/Neeraj/spark-2.3.0-bin-hadoop2.7/jars/graphframes-0.7.0-spark2.3-s_2.11.jar')
sqlcontext=SQLContext(sc)

In [6]:
station= sqlcontext.read.format("csv").option("header", "true").csv('201508_station_data.csv')
trip= sqlcontext.read.format("csv").option("header", "true").csv('201508_trip_data.csv')

In [7]:
station.show()

+----------+--------------------+---------+-----------+---------+------------+------------+
|station_id|                name|      lat|       long|dockcount|    landmark|installation|
+----------+--------------------+---------+-----------+---------+------------+------------+
|         2|San Jose Diridon ...|37.329732|-121.901782|       27|    San Jose|    8/6/2013|
|         3|San Jose Civic Ce...|37.330698|-121.888979|       15|    San Jose|    8/5/2013|
|         4|Santa Clara at Al...|37.333988|-121.894902|       11|    San Jose|    8/6/2013|
|         5|    Adobe on Almaden|37.331415|  -121.8932|       19|    San Jose|    8/5/2013|
|         6|    San Pedro Square|37.336721|-121.894074|       15|    San Jose|    8/7/2013|
|         7|Paseo de San Antonio|37.333798|-121.886943|       15|    San Jose|    8/7/2013|
|         8| San Salvador at 1st|37.330165|-121.885831|       15|    San Jose|    8/5/2013|
|         9|           Japantown|37.348742|-121.894715|       15|    San Jose|  

In [8]:
# station.printSchema

In [9]:
trip.show()

+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
|Trip ID|Duration|     Start Date|       Start Station|Start Terminal|       End Date|         End Station|End Terminal|Bike #|Subscriber Type|Zip Code|
+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
| 913460|     765|8/31/2015 23:26|Harry Bridges Pla...|            50|8/31/2015 23:39|San Francisco Cal...|          70|   288|     Subscriber|    2139|
| 913459|    1036|8/31/2015 23:11|San Antonio Shopp...|            31|8/31/2015 23:28|Mountain View Cit...|          27|    35|     Subscriber|   95032|
| 913455|     307|8/31/2015 23:13|      Post at Kearny|            47|8/31/2015 23:18|   2nd at South Park|          64|   468|     Subscriber|   94107|
| 913454|     409|8/31/2015 23:10|  San Jose City Hall|            10|8/31/2015 23

In [10]:
# station

In [11]:
import pyspark.sql.functions as sf
station.select(sf.concat("lat",sf.lit(" "),"long")).show(3)

+--------------------+
|concat(lat,  , long)|
+--------------------+
|37.329732 -121.90...|
|37.330698 -121.88...|
|37.333988 -121.89...|
+--------------------+
only showing top 3 rows



In [12]:
stationvertices=station.withColumnRenamed("name","id").select("id").distinct()
stationvertices.show(3)

+--------------------+
|                  id|
+--------------------+
|       2nd at Folsom|
|California Ave Ca...|
|Washington at Kea...|
+--------------------+
only showing top 3 rows



In [13]:
tripedges=trip.withColumnRenamed("Start Station","src")\
.withColumnRenamed("End Station","dst")\
.select("src","dst").groupBy("src","dst").count()
tripedges.show(3)

+--------------------+--------------------+-----+
|                 src|                 dst|count|
+--------------------+--------------------+-----+
|       2nd at Folsom|San Francisco Cal...|  179|
| Golden Gate at Polk|   Steuart at Market|   40|
|Grant Avenue at C...|Mechanics Plaza (...|   32|
+--------------------+--------------------+-----+
only showing top 3 rows



In [14]:
g=GraphFrame(stationvertices,tripedges)

In [15]:
g

GraphFrame(v:[id: string], e:[src: string, dst: string ... 1 more field])

In [16]:
g.vertices.show()

+--------------------+
|                  id|
+--------------------+
|       2nd at Folsom|
|California Ave Ca...|
|Washington at Kea...|
|Powell at Post (U...|
| Golden Gate at Polk|
|Yerba Buena Cente...|
|   Market at Sansome|
|         MLK Library|
|     Spear at Folsom|
|           Japantown|
|Commercial at Mon...|
|Paseo de San Antonio|
| San Salvador at 1st|
|Rengstorff Avenue...|
|     Townsend at 7th|
|Civic Center BART...|
|         Ryland Park|
|San Jose Diridon ...|
|San Jose Civic Ce...|
|     Post at Kearney|
+--------------------+
only showing top 20 rows



In [17]:
g.edges.show()

+--------------------+--------------------+-----+
|                 src|                 dst|count|
+--------------------+--------------------+-----+
|       2nd at Folsom|San Francisco Cal...|  179|
| Golden Gate at Polk|   Steuart at Market|   40|
|Grant Avenue at C...|Mechanics Plaza (...|   32|
|Temporary Transba...|Harry Bridges Pla...|   49|
|University and Em...|California Ave Ca...|   21|
|Evelyn Park and Ride|Evelyn Park and Ride|   12|
|   2nd at South Park|Yerba Buena Cente...|   16|
|      Market at 10th|      Market at 10th|   50|
|Grant Avenue at C...|Powell at Post (U...|   39|
|      Market at 10th|     Clay at Battery|    3|
|San Francisco Cal...|   Steuart at Market|  177|
|     Clay at Battery|Mechanics Plaza (...|   12|
|       2nd at Folsom|       Howard at 2nd|   18|
|Broadway St at Ba...|   2nd at South Park|   10|
|Embarcadero at Br...|    Davis at Jackson|   14|
|Embarcadero at Va...|       Howard at 2nd|   12|
|Commercial at Mon...|Mechanics Plaza (...|   36|


In [18]:
g.inDegrees.show()

+--------------------+--------+
|                  id|inDegree|
+--------------------+--------+
|       2nd at Folsom|      33|
|California Ave Ca...|       8|
|Powell at Post (U...|      35|
| Golden Gate at Polk|      35|
|Yerba Buena Cente...|      35|
|   Market at Sansome|      35|
|     Spear at Folsom|      35|
|         MLK Library|      14|
|           Japantown|      14|
|Commercial at Mon...|      35|
|Paseo de San Antonio|      15|
|Rengstorff Avenue...|       8|
| San Salvador at 1st|      13|
|     Townsend at 7th|      35|
|Civic Center BART...|      35|
|         Ryland Park|      11|
|San Jose Diridon ...|      16|
|San Jose Civic Ce...|       8|
|                null|       1|
|          Mezes Park|       2|
+--------------------+--------+
only showing top 20 rows



In [19]:
g.outDegrees.show()

+--------------------+---------+
|                  id|outDegree|
+--------------------+---------+
|       2nd at Folsom|       35|
|California Ave Ca...|        7|
|Powell at Post (U...|       35|
| Golden Gate at Polk|       33|
|Yerba Buena Cente...|       34|
|   Market at Sansome|       35|
|     Spear at Folsom|       35|
|         MLK Library|       14|
|           Japantown|       13|
|Commercial at Mon...|       35|
|Paseo de San Antonio|       14|
| San Salvador at 1st|       10|
|Rengstorff Avenue...|        9|
|     Townsend at 7th|       35|
|Civic Center BART...|       35|
|         Ryland Park|        9|
|San Jose Diridon ...|       16|
|San Jose Civic Ce...|       12|
|          Mezes Park|        4|
|SJSU - San Salvad...|       13|
+--------------------+---------+
only showing top 20 rows



In [20]:
g.find("(a)-[e]->(b)").show(truncate=False)

+-----------------------------------------------+-----------------------------------------------------------------------------------------+-----------------------------------------------+
|a                                              |e                                                                                        |b                                              |
+-----------------------------------------------+-----------------------------------------------------------------------------------------+-----------------------------------------------+
|[2nd at Folsom]                                |[2nd at Folsom, San Francisco Caltrain (Townsend at 4th), 179]                           |[San Francisco Caltrain (Townsend at 4th)]     |
|[Golden Gate at Polk]                          |[Golden Gate at Polk, Steuart at Market, 40]                                             |[Steuart at Market]                            |
|[Grant Avenue at Columbus Avenue]              |[Grant Aven

In [43]:
g.degrees.show()

+--------------------+------+
|                  id|degree|
+--------------------+------+
|       2nd at Folsom|    68|
|California Ave Ca...|    15|
|Powell at Post (U...|    70|
| Golden Gate at Polk|    68|
|Yerba Buena Cente...|    69|
|   Market at Sansome|    70|
|     Spear at Folsom|    70|
|         MLK Library|    28|
|           Japantown|    27|
|Commercial at Mon...|    70|
|Paseo de San Antonio|    29|
| San Salvador at 1st|    23|
|Rengstorff Avenue...|    17|
|     Townsend at 7th|    70|
|Civic Center BART...|    70|
|         Ryland Park|    20|
|San Jose Diridon ...|    32|
|San Jose Civic Ce...|    20|
|          Mezes Park|     6|
|                null|     1|
+--------------------+------+
only showing top 20 rows



In [34]:
g.inDegrees.sort('inDegree',ascending=False).show()

+--------------------+--------+
|                  id|inDegree|
+--------------------+--------+
|Mechanics Plaza (...|      35|
|Harry Bridges Pla...|      35|
|  Powell Street BART|      35|
|      Market at 10th|      35|
|       5th at Howard|      35|
|   Steuart at Market|      35|
|Washington at Kearny|      35|
|Embarcadero at Fo...|      35|
|     Clay at Battery|      35|
|     Beale at Market|      35|
|Commercial at Mon...|      35|
| Golden Gate at Polk|      35|
|     Spear at Folsom|      35|
|     Townsend at 7th|      35|
|   Market at Sansome|      35|
|Civic Center BART...|      35|
|Yerba Buena Cente...|      35|
|       Market at 4th|      35|
|Temporary Transba...|      35|
|Powell at Post (U...|      35|
+--------------------+--------+
only showing top 20 rows



In [46]:
# import pyspark.sql.functions as col
# g.inDegrees.join(g.outDegrees, g.inDegrees.select("id") === g.outDegrees.select("id"))\
#   .drop(g.outDegrees.select("id"))\
#   .selectExpr("id", "double(inDegree)/double(outDegree) as degreeRatio")